In [46]:
dataset = "mimic3"
task = "drugrec"
kg = "GPT-KG"

In [47]:
from graphcare import *

sample_dataset, G, ent2id, rel2id, ent_emb, rel_emb, \
            map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_rel_inv, \
                ccscm_id2clus, ccsproc_id2clus, atc3_id2clus = load_everything(dataset, task, kg, kg_ratio=1.0)

In [48]:
mode, out_channels, loss_function = get_mode_and_out_channels_and_loss_func(task=task, sample_dataset=sample_dataset)

# label direct ehr node
print("Labeling direct ehr nodes...")
sample_dataset = label_ehr_nodes(task, sample_dataset, len(map_cluster), ccscm_id2clus, ccsproc_id2clus, atc3_id2clus)
# sample_dataset = label_k_hop_nodes(G=G_tg, dataset=sample_dataset, k=2)
print("Splitting dataset...")
# train_dataset, val_dataset, test_dataset = split_by_patient(sample_dataset, [0.8, 0.1, 0.1], train_ratio=1.0, seed=528)

Labeling direct ehr nodes...


100%|██████████| 44399/44399 [00:01<00:00, 40232.14it/s]

Splitting dataset...


In [49]:
import random

ratio = 0.3
node_frac = 1- ratio

nodes = list(G.nodes())
random.shuffle(nodes)

num_nodes_to_remove = int(node_frac * G.number_of_nodes())
nodes_to_remove = nodes[:num_nodes_to_remove]

for node in nodes_to_remove:
    G.remove_node(node)
G_tg = from_networkx(G)

In [50]:
y_to_x_map = {}
for i, y in enumerate(G_tg.y):
    y_to_x_map[int(y)] = i

In [51]:
from copy import deepcopy

for patient in tqdm(sample_dataset):
    node_list = []
    for node in patient['node_set']:
        if node not in nodes_to_remove:
            node_list.append(node)

    patient['node_set'] = [y_to_x_map[y] for y in node_list]

100%|██████████| 44399/44399 [04:15<00:00, 173.86it/s]


In [52]:
all_set = Dataset(G=G_tg, dataset=sample_dataset, task=task)

In [53]:
from tqdm import tqdm

nodes_count = 0
edges_count = 0

for i in tqdm(range(len(all_set))):
    nodes_count += all_set[i].x.shape[0]
    edges_count += all_set[i].edge_index.shape[1]

avg_nodes = nodes_count / len(all_set)
avg_edges = edges_count / len(all_set)
print("avg_nodes: ", avg_nodes)
print("avg_edges: ", avg_edges)

 77%|███████▋  | 34282/44399 [01:36<00:28, 356.65it/s]


IndexError: index 826 is out of bounds for dimension 0 with size 826

In [54]:
avg_nodes = nodes_count / 34282
avg_edges = edges_count / 34282
print("avg_nodes: ", avg_nodes)
print("avg_edges: ", avg_edges)

avg_nodes:  139.7239950994691
avg_edges:  139.58946385858468
